<a href="https://colab.research.google.com/github/mjs1995/study/blob/main/spark/Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 48.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=a49243ec374de22ca536fc08f62d07c6ab0c07f8b7534cc4e5db473793ed5840
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [234]:
from pyspark.sql import functions as sf
from pyspark.sql.functions import substring
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

df = spark.createDataFrame([('ab001', 'MEM0089', 10),
                            ('ab001', 'MEM0043', 10),
                            ('ab001', 'MEM0019', 10),
                            ('ab001', 'MEM0019', 20),
                            ('ab001', 'MEM0019', 10),
                            ('ab002', 'MEM0027', 10),
                            ('ab002', 'MEM0027', 20),
                            ('ab002', 'MEM0027', 10),
                            ('ab002', 'MEM0004', 10),
                            ('ab002', 'MEM0004', 20),
                            ('ab002', 'MEM0065', 10),
                            ('ab002', 'MEM0065', 10),
                            ('ab002', 'MEM0011', 10),
                            ('ab002', 'MEM0011', 10),
                            ('ab002', 'MEM0011', 20),
                            ('ab002', 'MEM0011', 10),
                            ('ab002', 'MEM0011', 20),
                            ('ab002', 'MEM0011', 10),
                            ('ab002', 'MEM0082', 10),
                            ('ab002', 'MEM0082', 10)
                            ],
                           ["AB_ID", "MEM_ID", "ORD"])
df.show()

+-----+-------+---+
|AB_ID| MEM_ID|ORD|
+-----+-------+---+
|ab001|MEM0089| 10|
|ab001|MEM0043| 10|
|ab001|MEM0019| 10|
|ab001|MEM0019| 20|
|ab001|MEM0019| 10|
|ab002|MEM0027| 10|
|ab002|MEM0027| 20|
|ab002|MEM0027| 10|
|ab002|MEM0004| 10|
|ab002|MEM0004| 20|
|ab002|MEM0065| 10|
|ab002|MEM0065| 10|
|ab002|MEM0011| 10|
|ab002|MEM0011| 10|
|ab002|MEM0011| 20|
|ab002|MEM0011| 10|
|ab002|MEM0011| 20|
|ab002|MEM0011| 10|
|ab002|MEM0082| 10|
|ab002|MEM0082| 10|
+-----+-------+---+



In [243]:
new =     df.withColumn('MEM_SCT_CD1', sf.when(sf.col('ORD') == 10, sf.col('MEM_ID')).otherwise(''))\
            .withColumn("MEM_ID_1",sf.array(sf.col('MEM_SCT_CD1')))\
            .withColumn('MEM_SCT_CD', sf.when(sf.col('ORD') != 10, sf.col('MEM_ID')).otherwise(''))\
            .withColumn("MEM_ID_2",sf.array(sf.col('MEM_SCT_CD')))

In [244]:
new.show()

+-----+-------+---+-----------+---------+----------+---------+
|AB_ID| MEM_ID|ORD|MEM_SCT_CD1| MEM_ID_1|MEM_SCT_CD| MEM_ID_2|
+-----+-------+---+-----------+---------+----------+---------+
|ab001|MEM0089| 10|    MEM0089|[MEM0089]|          |       []|
|ab001|MEM0043| 10|    MEM0043|[MEM0043]|          |       []|
|ab001|MEM0019| 10|    MEM0019|[MEM0019]|          |       []|
|ab001|MEM0019| 20|           |       []|   MEM0019|[MEM0019]|
|ab001|MEM0019| 10|    MEM0019|[MEM0019]|          |       []|
|ab002|MEM0027| 10|    MEM0027|[MEM0027]|          |       []|
|ab002|MEM0027| 20|           |       []|   MEM0027|[MEM0027]|
|ab002|MEM0027| 10|    MEM0027|[MEM0027]|          |       []|
|ab002|MEM0004| 10|    MEM0004|[MEM0004]|          |       []|
|ab002|MEM0004| 20|           |       []|   MEM0004|[MEM0004]|
|ab002|MEM0065| 10|    MEM0065|[MEM0065]|          |       []|
|ab002|MEM0065| 10|    MEM0065|[MEM0065]|          |       []|
|ab002|MEM0011| 10|    MEM0011|[MEM0011]|          |   

In [203]:
new.groupBy('AB_ID').agg(
    (sf.countDistinct(sf.when(sf.col("ORD") == '10',sf.col("MEM_ID"))) 
    - sf.countDistinct(sf.when(sf.col("ORD") != '10',sf.col("MEM_ID")))).alias("PURE_ORD_MBR_CNT")
).show()

+-----+----------------+
|AB_ID|PURE_ORD_MBR_CNT|
+-----+----------------+
|ab002|               2|
|ab001|               2|
+-----+----------------+



In [204]:
new.groupBy('AB_ID').agg(
    (sf.countDistinct(sf.when(sf.col("ORD") == '10',sf.col("MEM_ID"))))).show()

+-----+-------------------------------------------+
|AB_ID|count(CASE WHEN (ORD = 10) THEN MEM_ID END)|
+-----+-------------------------------------------+
|ab002|                                          5|
|ab001|                                          3|
+-----+-------------------------------------------+



In [245]:
from pyspark.sql.functions import collect_list

ab = new.groupBy("AB_ID")\
    .agg(collect_list("MEM_SCT_CD1").alias("l1"),
         collect_list("MEM_SCT_CD").alias("l2")
        )
ab.show()

+-----+--------------------+--------------------+
|AB_ID|                  l1|                  l2|
+-----+--------------------+--------------------+
|ab002|[MEM0027, , MEM00...|[, MEM0027, , , M...|
|ab001|[MEM0089, MEM0043...|   [, , , MEM0019, ]|
+-----+--------------------+--------------------+



In [246]:
from collections import Counter

Counter(ab.select('l1').rdd.map(lambda x: [i for i in x[0]]).reduce(lambda x, y: x + y))

Counter({'': 5,
         'MEM0004': 1,
         'MEM0011': 4,
         'MEM0019': 2,
         'MEM0027': 2,
         'MEM0043': 1,
         'MEM0065': 2,
         'MEM0082': 2,
         'MEM0089': 1})

In [247]:
Counter(ab.select('l2').rdd.map(lambda x: [i for i in x[0]]).reduce(lambda x, y: x + y))

Counter({'': 15, 'MEM0004': 1, 'MEM0011': 2, 'MEM0019': 1, 'MEM0027': 1})

In [248]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from collections import Counter
from pyspark.sql.functions import col, lit

def sel(ab,x,y):
    return len(Counter(ab.select(x).rdd.map(lambda x: [i for i in x[0]]).reduce(lambda x, y: x + y)) - Counter(ab.select(y).rdd.map(lambda x: [i for i in x[0]]).reduce(lambda x, y: x + y))) 

sel_cols = udf(sel, IntegerType())

In [249]:
ab1 = ab.filter(sf.col("AB_ID") == 'ab001')
ab1 = ab1.withColumn('res',lit(sel(ab1,'l1', 'l2')))

In [250]:
ab2 = ab.filter(sf.col("AB_ID") == 'ab002')
ab2 = ab2.withColumn('res',lit(sel(ab2,'l1', 'l2')))

In [251]:
new.filter(sf.col("AB_ID") == 'ab002').show()

+-----+-------+---+-----------+---------+----------+---------+
|AB_ID| MEM_ID|ORD|MEM_SCT_CD1| MEM_ID_1|MEM_SCT_CD| MEM_ID_2|
+-----+-------+---+-----------+---------+----------+---------+
|ab002|MEM0027| 10|    MEM0027|[MEM0027]|          |       []|
|ab002|MEM0027| 20|           |       []|   MEM0027|[MEM0027]|
|ab002|MEM0027| 10|    MEM0027|[MEM0027]|          |       []|
|ab002|MEM0004| 10|    MEM0004|[MEM0004]|          |       []|
|ab002|MEM0004| 20|           |       []|   MEM0004|[MEM0004]|
|ab002|MEM0065| 10|    MEM0065|[MEM0065]|          |       []|
|ab002|MEM0065| 10|    MEM0065|[MEM0065]|          |       []|
|ab002|MEM0011| 10|    MEM0011|[MEM0011]|          |       []|
|ab002|MEM0011| 10|    MEM0011|[MEM0011]|          |       []|
|ab002|MEM0011| 20|           |       []|   MEM0011|[MEM0011]|
|ab002|MEM0011| 10|    MEM0011|[MEM0011]|          |       []|
|ab002|MEM0011| 20|           |       []|   MEM0011|[MEM0011]|
|ab002|MEM0011| 10|    MEM0011|[MEM0011]|          |   

In [252]:
Counter(ab2.select('l1').rdd.map(lambda x: [i for i in x[0]]).reduce(lambda x, y: x + y))

Counter({'': 4,
         'MEM0004': 1,
         'MEM0011': 4,
         'MEM0027': 2,
         'MEM0065': 2,
         'MEM0082': 2})

In [253]:
Counter(ab2.select('l2').rdd.map(lambda x: [i for i in x[0]]).reduce(lambda x, y: x + y))

Counter({'': 11, 'MEM0004': 1, 'MEM0011': 2, 'MEM0027': 1})

In [254]:
ab2.show()

+-----+--------------------+--------------------+---+
|AB_ID|                  l1|                  l2|res|
+-----+--------------------+--------------------+---+
|ab002|[MEM0027, , MEM00...|[, MEM0027, , , M...|  4|
+-----+--------------------+--------------------+---+



In [255]:
ab1.union(ab2).select('AB_ID','res')\
   .withColumnRenamed('AB_ID','상품ID')\
   .withColumnRenamed('res','순주문수').show()

+------+--------+
|상품ID|순주문수|
+------+--------+
| ab001|       3|
| ab002|       4|
+------+--------+

